### Nucleotide Frequency & 3-mer composition

In [ ]:
from Bio import SeqIO
from collections import Counter
from itertools import product
from Bio.SeqUtils import GC
import pandas as pd

blast_dir = '/disk8/3.YS/20.ML/07.blast/02.blastx_result'

def calculate_nucleotide_frequency(sequence):
    return Counter(sequence)

def initialize_kmer_dict(k):
    kmers = [''.join(p) for p in product('ATGC', repeat=k)]
    return {kmer: 0 for kmer in kmers}

def calculate_kmer_composition(sequence, k):
    kmer_counts = initialize_kmer_dict(k)
    
    actual_kmer_counts = Counter([sequence[i:i+k] for i in range(len(sequence) - k + 1)])
    
    for kmer, count in actual_kmer_counts.items():
        if kmer in kmer_counts:
            kmer_counts[kmer] = count

    return kmer_counts

def calculate_gc_content(sequence):    
    return GC(sequence)

def check_hgt(gcf, record_id):
    try:
        df = pd.read_csv(f'{blast_dir}/{gcf}.out', sep='\t', index_col=None, header=None)
        hgt_list = df.iloc[:, 0].tolist()
        if record_id in hgt_list:
            return 1
        else:
            return 0
    except FileNotFoundError:
        print(f"Warning: File {gcf}.out not found.")
        return 0
    
def main(cds_file):
    gcf = cds_file.split('/')[-1].split(".")[0]
    for record in SeqIO.parse(cds_file, "fasta"):
        sequence = str(record.seq)
        hgt_boolean = check_hgt(gcf, record.id)
        nucleotide_freq = calculate_nucleotide_frequency(sequence)
        dinucleotide_freq = calculate_kmer_composition(sequence, 2)  # 2-mer (dinucleotide)
        trinucleotide_freq = calculate_kmer_composition(sequence, 3)  # 3-mer (trinucleotide)
        gc_content = calculate_gc_content(sequence)

        # Output the results for each CDS
        print(f"Results for CDS {record.id}:")
        print("Is HGT?: ", hgt_boolean)
        print("Nucleotide Frequency:", nucleotide_freq)
        print("2-mer Frequency:", dinucleotide_freq)
        print("3-mer Frequency:", trinucleotide_freq)
        print(f"GC Content: {gc_content:.2f}%")
        print("\n")

#        with open (f'/disk8/3.YS/20.ML/00.data/nucleotide_info/{gcf}.txt', 'a') as final:
#            final.write(record.id+'\t')
#            final.write(str(hgt_boolean) +'\t')
#            final.write(str(nucleotide_freq['A'])+'\t')
#            final.write(str(nucleotide_freq['T'])+'\t')
#            final.write(str(nucleotide_freq['G'])+'\t')
#            final.write(str(nucleotide_freq['C'])+'\t')
#            for two_mer in dinucleotide_freq:
#                final.write(str(dinucleotide_freq[two_mer]) +'\t')
#            for three_mer in trinucleotide_freq:
#                final.write(str(trinucleotide_freq[three_mer]) +'\t')
#            final.write(str(gc_content) +'\n')
                


In [55]:
import os
from multiprocessing import Pool
cds_dir = '/disk8/3.YS/20.ML/03.prodigal'
cds_files = os.listdir(cds_dir)
cds_files = [cds_dir+'/'+a for a in cds_files]

In [56]:
cds_files[:3]

['/disk8/3.YS/20.ML/03.prodigal/GCF_000513595.CDS',
 '/disk8/3.YS/20.ML/03.prodigal/GCF_002813775.CDS',
 '/disk8/3.YS/20.ML/03.prodigal/GCF_014334175.CDS']

In [ ]:
if __name__ == "__main__":
    data = cds_files

    with Pool(processes=100) as pool:
        pool.map(main, data)